In [1]:
"""
GENERAL STEPS
- run sql query to get all non-spatial TMC data and load into dataframe
- load shp of NHS segments into geodataframe
- left join non-spatial TMC table to NHS geotable
- where spatial data missing:
    - add "true_shp" flag; set to 0 (1 = has true shape, 0 = needs true shape built)
    - compute "stick" geometry using start/end lat/long vals.
    - CHECK TO DO: can you programmatically check if a TMC's start/end points haven't changed, and if not just plug in the geom from the 2021 INRIX file?
 - export to feature class; DAMS will need to manually correct missing true shapes (how to prioritize?)
    
"""

import geopandas as gpd
import pandas as pd

from esri_file_to_dataframe import esri_to_df

# true-shape TMCs, but only for NHS
shp_tmc_nhs = r"I:\Projects\Darren\PPA3_GIS\PPA3_GIS.gdb\NPMRDS_2023_NHS_SACOG" # on WIN10-MODEL-2



# load shp to gdf
shp_fields = ['Tmc']
shp_crs = arcpy.Describe(shp_tmc_nhs).spatialReference.factoryCode # 2226 = EPSG code for SACOG region
gdf_nhs = esri_to_df(esri_obj_path=shp_tmc_nhs, include_geom=True, field_list=shp_fields, index_field=None, 
               crs_val=crs_sacog, dissolve=False)



In [ ]:
# non-spatial data stored in SQL Server
npmrds_db = 'NPMRDS'
tt_tbl = 'npmrds_2023_alltmc_paxtruck_comb'
tmc_txt_tbl = 'npmrds_2023_alltmc_txt' # full TMC network, but lacking spatial true-shape data
crs_npmrds_raw = 4326 # WGS 84, the default system for the lat-lon points


# run query and load to spatial df
print("running query for speed data...")

with open('PPA_NPMRDS_metrics_latest.sql', 'w') as f:
    sql_template = f.read()
    
    params = dict(tt_tbl=tt_tbl, tmc_tbl=tmc_txt_tbl)
    formatted_sql = sql_template.format(**params)

df_npmrds = sqlqry_to_df(query_str=formatted_sql, dbname=npmrds_db)